In [1]:
# import pandas as pd
# asteoroids = pd.read_csv('datasets/Asteroid Selection.csv')
# asteoroids.drop(columns=['Spec.type','Launch date (one-way)','Min total dv (km/s)'],inplace=True)
# asteoroids.rename(columns={'Launch date (round-trip)': 'Launch date'}, inplace=True)
# asteoroids.to_csv('datasets/Asteroid.csv', index=False)
# asteoroids

In [2]:
import pandas as pd
asteoroids = pd.read_csv('datasets/Asteroid.csv')

In [3]:
asteoroids

,Spkid,Surname,Name,OCC,Diameter (km),TOF (days),Launch date,Min total dv (km/s).1,Mission duration (days),Stay (days)
0,2000433,1898 DQ,Eros,0,"16,84",270,2025-08-13,14.41,348,8
1,2000719,1911 MT,Albert,0,"2,71 ÷ 6.07",180,2048-04-22,23.15,448,8
2,2000887,1918 DB,Alinda,0,"4,2",420,2025-01-01,20.59,388,8
3,2001036,1924 TD,Ganymed,0,"37,675",160,2037-05-18,31.36,348,8
4,2001580,1950 KA,Betulia,0,"5,8",440,2027-12-09,40.73,350,10
...,...,...,...,...,...,...,...,...,...,...
321,3114075,2002 DO3,NaN,6,"0,14 ÷ 0.31",310,2039-07-17,11.46,338,8
322,3114077,2002 DQ3,NaN,6,"0,06 ÷ 0.13",380,2032-06-13,10.93,443,8
323,3114105,2002 EA,NaN,8,"0,11 ÷ 0.25",120,2030-09-02,15.00,353,8
324,3114106,2002 EC,NaN,7,"0,07 ÷ 0.16",430,2042-09-29,13.27,173,8


In [4]:
# asteoroids.drop(columns=['Spkid','Surname'], inplace=True)

In [5]:
asteoroids.rename(columns={'Diameter (km)': 'Diameter', 'Min total dv (km/s).1': 'Min_dv', 'Launch date': 'Launch_date', 'Mission duration (days)': 'Duration', 'Stay (days)': 'Stay', 'TOF (days)': 'TOF'}, inplace=True)

In [6]:
import re

def replace_range_with_middle(text):
    match = re.search(r'(\d+(\,\d+)?) ÷ (\d+(\.\d+)?)', text)
    if match:
        num1 = float(match.group(1).replace(',','.'))
        num2 = float(match.group(3))
        middle = (num1 + num2) / 2
        return float(text.replace(match.group(0), str(round(middle, 2))))
    return float(text.replace(',','.'))

asteoroids['Diameter'] = asteoroids['Diameter'].apply(replace_range_with_middle)

In [7]:
asteoroids

,Spkid,Surname,Name,OCC,Diameter,TOF,Launch_date,Min_dv,Duration,Stay
0,2000433,1898 DQ,Eros,0,16.840,270,2025-08-13,14.41,348,8
1,2000719,1911 MT,Albert,0,4.390,180,2048-04-22,23.15,448,8
2,2000887,1918 DB,Alinda,0,4.200,420,2025-01-01,20.59,388,8
3,2001036,1924 TD,Ganymed,0,37.675,160,2037-05-18,31.36,348,8
4,2001580,1950 KA,Betulia,0,5.800,440,2027-12-09,40.73,350,10
...,...,...,...,...,...,...,...,...,...,...
321,3114075,2002 DO3,NaN,6,0.230,310,2039-07-17,11.46,338,8
322,3114077,2002 DQ3,NaN,6,0.100,380,2032-06-13,10.93,443,8
323,3114105,2002 EA,NaN,8,0.180,120,2030-09-02,15.00,353,8
324,3114106,2002 EC,NaN,7,0.120,430,2042-09-29,13.27,173,8


In [8]:
asteoroids['Launch_date'] = pd.to_datetime(asteoroids['Launch_date'], format='%Y-%m-%d')

In [9]:
import sqlite3

conn = sqlite3.connect("database.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS ASTEROIDS;")
cursor.execute("""
CREATE TABLE ASTEROIDS (
    Spkid INTEGER NOT NULL,
    Surname TEXT NOT NULL,
    Name TEXT,
    OCC INTEGER NOT NULL,
    Diameter REAL NOT NULL,
    TOF REAL NOT NULL,
    Launch_date DATE NOT NULL,
    Min_dv REAL NOT NULL,
    Duration INTEGER NOT NULL,
    Stay INTEGER NOT NULL
)
""")
print("Table created successfully.")

asteoroids.to_sql("ASTEROIDS", conn, if_exists="append", index=False)
print("Data inserted successfully.")

conn.commit()
conn.close()

Table created successfully.
Data inserted successfully.


In [10]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database.db")
cursor = conn.cursor()
query = "SELECT * FROM ASTEROIDS;"
df = pd.read_sql_query(query, conn)
df

,Spkid,Surname,Name,OCC,Diameter,TOF,Launch_date,Min_dv,Duration,Stay
0,2000433,1898 DQ,Eros,0,16.840,270.0,2025-08-13 00:00:00,14.41,348,8
1,2000719,1911 MT,Albert,0,4.390,180.0,2048-04-22 00:00:00,23.15,448,8
2,2000887,1918 DB,Alinda,0,4.200,420.0,2025-01-01 00:00:00,20.59,388,8
3,2001036,1924 TD,Ganymed,0,37.675,160.0,2037-05-18 00:00:00,31.36,348,8
4,2001580,1950 KA,Betulia,0,5.800,440.0,2027-12-09 00:00:00,40.73,350,10
...,...,...,...,...,...,...,...,...,...,...
321,3114075,2002 DO3,None,6,0.230,310.0,2039-07-17 00:00:00,11.46,338,8
322,3114077,2002 DQ3,None,6,0.100,380.0,2032-06-13 00:00:00,10.93,443,8
323,3114105,2002 EA,None,8,0.180,120.0,2030-09-02 00:00:00,15.00,353,8
324,3114106,2002 EC,None,7,0.120,430.0,2042-09-29 00:00:00,13.27,173,8
